Demo is running on Python 3.7.7

In [1]:
import tellurium as te
import numpy as np

In [2]:
from doe_tool import *

In [3]:
START = 0
END = 100
NUM_PTS = 100
PAR_NAMES = ('alpha', 'beta')
IN_VALS = (0.1, 1)

Suppose we have a simple model of single interaction dynamics

In [4]:
model = '''
$X->Y; beta*X;
Y-> ; alpha*Y;

X = 1; Y = 0;
beta = 1;
alpha = 0.1;

'''

r = te.loada(model)

function below represents what will be done automatically in the finished scanner

In [5]:
def model(sim_specs):
    r.resetAll()
    for key in sim_specs:
        r[key] = sim_specs[key]
    return r.simulate(START, END, NUM_PTS)

The finished product will start here

In [6]:
g = doe_tool(model, PAR_NAMES, IN_VALS)

Once the model is loaded, a simulator coroutine is created and calls to it are
performed internally like so:

In [7]:
test = g.simulate({'alpha': .1, 'beta': 1})
test

       time,      [Y]
 [[       0,        0],
  [  1.0101, 0.960759],
  [  2.0202,  1.82921],
  [  3.0303,  2.61423],
  [  4.0404,  3.32383],
  [ 5.05051,  3.96525],
  [ 6.06061,  4.54505],
  [ 7.07071,  5.06914],
  [ 8.08081,  5.54288],
  [ 9.09091,  5.97111],
  [  10.101,  6.35819],
  [ 11.1111,  6.70808],
  [ 12.1212,  7.02435],
  [ 13.1313,  7.31023],
  [ 14.1414,  7.56865],
  [ 15.1515,  7.80224],
  [ 16.1616,   8.0134],
  [ 17.1717,  8.20426],
  [ 18.1818,  8.37679],
  [ 19.1919,  8.53274],
  [  20.202,   8.6737],
  [ 21.2121,  8.80113],
  [ 22.2222,  8.91631],
  [ 23.2323,  9.02043],
  [ 24.2424,  9.11454],
  [ 25.2525,  9.19961],
  [ 26.2626,  9.27651],
  [ 27.2727,  9.34602],
  [ 28.2828,  9.40885],
  [ 29.2929,  9.46564],
  [  30.303,  9.51698],
  [ 31.3131,  9.56339],
  [ 32.3232,  9.60533],
  [ 33.3333,  9.64325],
  [ 34.3434,  9.67752],
  [ 35.3535,   9.7085],
  [ 36.3636,  9.73651],
  [ 37.3737,  9.76182],
  [ 38.3838,   9.7847],
  [ 39.3939,  9.80539],
  [  40.404,  9.82

Extracting other values form the model that do not require simulation (such as Eigen values) is in a To Do list, but will similarly be calls to the coroutine

Post processing of Time Series data is expected to vary between cases, so the program wraps
any normal function into a coroutine:

In [8]:
def post_processing(data):
    # Example of finding response time of a model
    # extract last value in dataset, divide by 2 and find
    # location of closest value in dataset
    halfmax = (data['[Y]'][-1])/2
    return data['time'][np.argmin(abs(data['[Y]'] - halfmax))]

In [9]:
g.load_post_processor(post_processing)

Calls to the post processor are performed internally like so:

In [10]:
test_post = g.send_post_processor(test)
test_post

7.070707070707071

The actual scan with an optional search condition starts here

In [11]:
# Conditionals setup
g.set_conditional('boundary', 0, 2)

Internal calls are, again, sent like so:

In [12]:
g.send_conditional(20)

False

The actual scanning algorithm is under development, the placeholder prototype will only work for the gridsearch

In [13]:
grid_search_parameters = (
    ('alpha', [.01, .1, 1, 10, 100] ),
    ('beta', range(1, 5))
)
g.scan_demo(grid_search_parameters)

{'alpha': 1, 'beta': 1}

In [14]:
g.send_post_processor(g.simulate({'alpha': 1, 'beta': 1}))

1.0101010101010102

So, in the finished product, running the parameter scan for a model of interest will only take a few steps:

In [15]:
scanner = doe_tool(model, PAR_NAMES, IN_VALS) #load model
scanner.load_post_processor(post_processing) #setup post processor, if needed
scanner.set_conditional('boundary', 0, 2) #set up a condition of exiting scan
scanner.scan_demo(grid_search_parameters) #load in parameters of interest and start scanning

{'alpha': 1, 'beta': 1}

Getting the intermediate runs, or the complete scan is possible using the following command;
keep in mind that this will singnificantly slow down the scan

In [17]:
simulations, results, specs = scanner.get_simulations(grid_search_parameters)

In [18]:
print((results[0], results[5]))
print((specs[0], specs[5]))

(38.38383838383839, 7.070707070707071)
({'alpha': 0.01, 'beta': 1}, {'alpha': 0.1, 'beta': 2})
